# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [11]:
fact_sheet_chair = """
OVERVIEW
The Lamborghini Aventador is a flagship supercar from the renowned Italian automaker, Lamborghini. Known for its striking design, powerful performance, and advanced technology, the Aventador stands as a pinnacle of automotive engineering and luxury.

Key Specifications
Engine
Type: 6.5L V12 Naturally Aspirated
Power: 730 hp (Aventador S) / 759 hp (Aventador SVJ)
Torque: 509 lb-ft (Aventador S) / 531 lb-ft (Aventador SVJ)
Transmission: 7-speed ISR (Independent Shifting Rod) Automated Manual
Performance
Top Speed: 217 mph (350 km/h)
0-60 mph: 2.9 seconds (Aventador S) / 2.8 seconds (Aventador SVJ)
Drive Type: All-wheel drive

DIMENSIONS
Length: 190.4 inches (4,848 mm)
Width: 79.9 inches (2,030 mm) without mirrors
Height: 44.7 inches (1,136 mm)
Wheelbase: 106.3 inches (2,703 mm)
Curb Weight: 3,472 lbs (1,575 kg)


Special Editions
Aventador SVJ (Superveloce Jota)
Enhanced performance with increased power and reduced weight
Extensive use of carbon fiber for aerodynamic enhancements
Limited production run
Aventador S
Enhanced aerodynamics and suspension system
Improved driving dynamics with rear-wheel steering
Upgraded infotainment and interior features
Pricing
Base Price:
Aventador S: Starting at $421,321
Aventador SVJ: Starting at $517,770
"""

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing the Lamborghini Aventador, the epitome of luxury and performance in the world of supercars. Crafted by the prestigious Italian automaker, Lamborghini, the Aventador is a masterpiece of automotive engineering.

Powered by a 6.5L V12 Naturally Aspirated engine, the Aventador delivers an impressive 730 hp in the Aventador S model and a staggering 759 hp in the Aventador SVJ. With a torque of 509 lb-ft in the Aventador S and 531 lb-ft in the Aventador SVJ, this supercar is designed to provide an exhilarating driving experience.

Equipped with a 7-speed ISR (Independent Shifting Rod) Automated Manual transmission, the Aventador offers seamless gear shifts for optimal performance. With a top speed of 217 mph and a 0-60 mph acceleration time of just 2.8 seconds in the Aventador SVJ, this supercar is sure to leave a lasting impression.

The Aventador's dimensions showcase its sleek and aerodynamic design, with a length of 190.4 inches, a width of 79.9 inches (without mirrors), and 

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [13]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Experience the pinnacle of automotive luxury with the Lamborghini Aventador. Featuring a powerful 6.5L V12 engine, striking design, and top speed of 217 mph, this flagship supercar offers unparalleled performance. Choose from the Aventador S or SVJ models for enhanced features and limited edition exclusivity. Starting at $421,321.


In [14]:
len(response.split())

48

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [15]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for automobile lovers, 
so should be technical in nature and focus on the 
engine of the product.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience the pinnacle of automotive engineering with the Lamborghini Aventador. Featuring a 6.5L V12 engine, delivering 730 hp in the Aventador S and 759 hp in the Aventador SVJ, this supercar offers unparalleled power and performance. With a top speed of 217 mph and 0-60 mph in just 2.8 seconds, the Aventador is a true masterpiece for automobile enthusiasts.


In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for automobile lovers, 
so should be technical in nature and focus on the 
engine of the product.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience the raw power and precision of the Lamborghini Aventador, featuring a 6.5L V12 engine delivering up to 759 hp. With a top speed of 217 mph and 0-60 mph in just 2.8 seconds, this supercar is a true engineering marvel. Product ID: 1,575 kg.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [17]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

```The Lamborghini Aventador is a masterpiece of automotive engineering, crafted with precision and luxury in mind. This flagship supercar boasts a powerful 6.5L V12 Naturally Aspirated engine that delivers an impressive 730 hp in the Aventador S and 759 hp in the Aventador SVJ. The advanced 7-speed ISR Automated Manual transmission ensures seamless gear shifts, while the all-wheel drive system provides exceptional handling.

Constructed with the finest materials, the Aventador features extensive use of carbon fiber for aerodynamic enhancements, reducing weight and increasing performance. The sleek design of the Aventador is complemented by its impressive dimensions, including a length of 190.4 inches, a width of 79.9 inches (without mirrors), and a height of 44.7 inches. The wheelbase measures 106.3 inches, providing stability and agility on the road.

Experience the pinnacle of automotive luxury with the Lamborghini Aventador, where power, performance, and style converge to create a 

## Load Python libraries to view HTML

In [18]:
from IPython.display import display, HTML

In [19]:
display(HTML(response))

Length,190.4 inches
Width,79.9 inches
Height,44.7 inches
Wheelbase,106.3 inches
Curb Weight,"3,472 lbs"


## Try experimenting on your own!